This notebook will read in the prepped track files and remove tracks representing non-focal and non-zebra individuals.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# list of usable observations
focal_obs = ['015', '027', '066', '074']

In [3]:
for t in focal_obs:
    file = '../data/tracks/ob' + t + '_tracks.csv'
    tracks = pd.read_csv(file)
    clean_tracks = tracks[((tracks['spp'] == 'gz') | (tracks['spp'] == 'pz')) &
                          (tracks['focal'] == 1)]
    clean_tracks.dropna(inplace = True)
    name = 'ob' + t + '_clean.csv'
    clean_tracks.to_csv(os.path.join('../data', 'clean_tracks', name), index = False)

Each file now only includes the tracks for zebras that are also focal individuals.